In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/train.csv')
df = df[df["user_id"].isin([0,1,2,3])]

df = pd.read_csv("data/item_metadata.csv")

titles = df["title"].dropna()
all_words = [word for title in titles for word in title.split()]

uqw = set(all_words)
print(f"Number of unique words in title: {len(uqw)}")

Number of unique words in title: 219347


In [4]:
print(titles[0])
print(titles[1])
print(titles[2])

Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)
Yes to Tomatoes Detoxifying Charcoal Cleanser (Pack of 2) with Charcoal Powder, Tomato Fruit Extract, and Gingko Biloba Leaf Extract, 5 fl. oz.
Eye Patch Black Adult with Tie Band (6 Per Pack)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=50000)
X = vectorizer.fit_transform(df["title"].fillna(""))
print(X.shape[1])

50000


In [6]:
# Example using gensim's Word2Vec
from gensim.models import Word2Vec

titles = df["title"].dropna().apply(str.split).tolist()
stores = df["store"].dropna().apply(str.split).tolist()
description = df["description"].dropna().apply(str.split).tolist()
category = df["category"].dropna().apply(str.split).tolist()
model = Word2Vec(titles, vector_size=100, window=5, min_count=2, workers=4)

# Average vectors per title
def get_vector(title):
    words = title.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

df["title_v"] = df["title"].fillna("").apply(get_vector)
df["store_v"] = df["store"].fillna("").apply(get_vector)
df["description_v"] = df["description"].fillna("").apply(get_vector)
df["category_v"] = df["category"].fillna("").apply(get_vector)


In [34]:
inx = 262133 
print(df["category_v"][inx])
print(df["category"][inx])
print(df["category"].unique())

sims = model.wv.most_similar('Beauty', topn=10)
print(sims)

[ 0.79477525 -0.09145572 -0.9448379   1.4566729  -1.7657417  -2.0354183
 -0.2712507   0.40963253  0.6647276  -1.5204257  -1.2012249  -1.5572634
  1.3268585  -0.8547957   0.6263747  -2.7810667   2.1434586   0.9227878
 -0.8290316  -2.7154913  -1.6258346   0.87365144 -2.2809734   0.57719314
 -2.2244203   1.1003994  -1.2284206  -0.5892255  -4.2885036  -1.7923474
 -0.24968293 -1.2637568   0.08234332 -0.43256813  0.36135685 -0.9332717
 -1.530892    0.82903993 -2.8110201   0.5109619   2.171602    1.2829512
  0.562826    0.33634365  2.5344708   1.9127059  -3.1565902   0.5226751
 -0.65061295 -0.01731543  0.01171699  0.09372376  1.4024599  -0.31334814
 -2.0613742   2.2525854  -0.6618999  -0.10650872 -1.979642    1.3667437
 -1.3454782   0.17342056  0.49517506  0.8262319  -0.6465023  -0.3157715
  0.33003154  2.8523993   1.9423872   1.2163053  -1.7273448   0.84354424
  1.9067237   0.2039789   0.6218543  -1.2607628   0.3642694  -1.2623798
 -0.57700616 -0.03500407  0.7092313   0.5953518  -2.892473   

In [27]:
print(df.columns)
### KATEGORIE DO WEKTORYZACJI
"""
1. main_category
2. title
3. store
5. description
6. category
"""
#### WYRZUCIĆ
"""
0. features   -> narazie zróbmy uproszczoną wersje
1. categories -> narazie zróbmy uproszczoną wersje
2. image_count
3. has_images
4. image_urls
5. images
"""
# print(df.head())
val = df["title_v"][0]
print(val)
print(np.linalg.det(val))
print(df["category"][0])

Index(['parent_asin', 'main_category', 'title', 'average_rating',
       'rating_number', 'price', 'store', 'features', 'description', 'images',
       'categories', 'image_count', 'has_images', 'image_urls', 'category',
       'title_v', 'store_v', 'description_v', 'category_v'],
      dtype='object')
[-2.59720296e-01  1.68001652e-01  3.01649243e-01 -1.31225541e-01
  1.03785850e-01 -3.74051213e-01 -2.18160093e-01  7.75986671e-01
 -1.58428267e-01 -1.33072406e-01  1.06063254e-01 -3.73020500e-01
  1.26233324e-01  1.44248769e-01  5.96812069e-02  1.52890697e-01
  6.05553389e-03 -1.58345491e-01  3.34438193e-03 -2.50568718e-01
 -1.77389205e-01  2.16671005e-01 -3.77750732e-02  4.12624702e-02
 -3.69811989e-02 -1.30940095e-01 -3.21436822e-01 -2.10829720e-01
 -1.67468771e-01  2.91815013e-01  1.90468431e-01 -3.93933982e-01
 -8.69942382e-02 -2.15910062e-01 -3.56210060e-02 -1.72088921e-01
 -4.19497222e-01 -3.15286428e-01  1.53820753e-01 -7.16338158e-01
 -2.55102366e-01 -2.18074280e-03 -1.16263233e-

LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


high_cardinality_cols = ['parent_asin', 'main_category', 'image_urls', 'images', 'has_images','features', 'categories']

categorical_cols = [col for col in df.select_dtypes(include='object').columns
                    if col not in high_cardinality_cols]


df['title'] = df.apply(np.linalg.det, axis=1)
df['store'] = df.apply(np.linalg.det, axis=1)
df['description'] = df.apply(np.linalg.det, axis=1)
df['category'] = df.apply(np.linalg.det, axis=1)

# print(df['title'].apply(type))
# df = np.array(df['title'])
# object_cols = df.select_dtypes(include='object')
df_filtered = df[categorical_cols]

cosine_similarity = cosine_similarity(df_filtered)
# cosine_similarity = cosine_similarity(df[categorical_cols])
print(cosine_similarity)
similarity_df = pd.DataFrame(cosine_similarity, index=genre_df.index, columns=genre_df.index)


ValueError: setting an array element with a sequence.

In [ ]:

# STEP 3: Apply Bag of Words transformation to each categorical column
for col in categorical_cols:
    try:
        vectorizer = CountVectorizer(max_features=1000, stop_words=None)
        X = vectorizer.fit_transform(text_data)
        bow_df = pd.DataFrame(X.toarray(), columns=[f"{col}_{feat}" for feat in vectorizer.get_feature_names_out()])
        bow_features.append(bow_df)
    except ValueError as e:
        print(f"Skipping column '{col}' due to error: {e}")

    # Initialize CountVectorizer
    # vectorizer = CountVectorizer(stop_words='english')

    # Transform the column into BoW representation
    # X_bow = vectorizer.fit_transform(text_data)

    # Create a DataFrame from the result with meaningful column names
    # bow_df = pd.DataFrame(X_bow.toarray(), columns=[f"{col}_{feat}" for feat in vectorizer.get_feature_names_out()])

    # bow_features.append(bow_df)

# STEP 4: Concatenate all BoW DataFrames horizontally
bow_result = pd.concat(bow_features, axis=1)


# Optional: Add numerical columns if you want to retain them
numerical_cols = df.select_dtypes(include=['number']).reset_index(drop=True)
final_df = pd.concat([numerical_cols, bow_result], axis=1)

# Show result

Processing column: main_category
Processing column: title
Processing column: store
Processing column: description
Processing column: category


In [1]:
final_df

NameError: name 'final_df' is not defined